<a href="https://colab.research.google.com/github/BilalAsifB/CODEE/blob/main/CODEE_qwen_2_5_3_SFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import HfApi
from google.colab import userdata

api = HfApi()

token = userdata.get('HF_TOKEN')

print(api.whoami(token=token))

{'type': 'user', 'id': '6852e415a1646a48e7f75ce7', 'name': 'bilalburney', 'fullname': 'Bilal Asif Burney', 'email': 'bilalburney14@gmail.com', 'emailVerified': True, 'canPay': False, 'billingMode': 'postpaid', 'periodEnd': None, 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/no-auth/PUnRYx-8ajH5oGEm8ZMJ6.png', 'orgs': [{'type': 'org', 'id': '68edf9a10da63eef75514b89', 'name': 'nanochat-students', 'fullname': 'nanochat students', 'email': None, 'canPay': False, 'billingMode': 'postpaid', 'periodEnd': None, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/62d648291fa3e4e7ae3fa6e8/6Uk_2esDQItKgBunvl_nr.png', 'roleInOrg': 'contributor', 'isEnterprise': False}], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'notebooks', 'role': 'write', 'createdAt': '2025-11-25T19:10:54.274Z'}}}


In [2]:
!pip install transformers==4.55.2 peft==0.17.0 accelerate==1.10.0 trl==0.21.0 bitsandbytes==0.47.0 datasets==4.0.0 huggingface-hub==0.34.4 safetensors==0.6.2 pandas==2.2.2 matplotlib==3.10.0 numpy==2.0.2 --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.8/485.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 33.1 MB/s eta 0:00:00


In [3]:
import os
import torch
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer

In [4]:
bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.float32
)
repo_id = 'Qwen/Qwen2.5-Coder-3B-Instruct'
model = AutoModelForCausalLM.from_pretrained(
   repo_id, device_map="cuda:0", quantization_config=bnb_config
)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [5]:
print(model.get_memory_footprint()/1e6)

2010.079488


In [6]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((2048,), 

In [7]:
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=16,
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)
model = get_peft_model(model, config)

In [8]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 2048)
        (layers): ModuleList(
          (0-35): 36 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.L

In [9]:
print(model.get_memory_footprint()/1e6)

2692.759808


In [10]:
train_p, tot_p = model.get_nb_trainable_parameters()
print(f'Trainable parameters:      {train_p/1e6:.2f}M')
print(f'Total parameters:          {tot_p/1e6:.2f}M')
print(f'% of trainable parameters: {100*train_p/tot_p:.2f}%')

Trainable parameters:      14.97M
Total parameters:          3100.91M
% of trainable parameters: 0.48%


In [11]:
dataset = load_dataset("sahil2801/CodeAlpaca-20k", split="train")

README.md:   0%|          | 0.00/147 [00:00<?, ?B/s]

code_alpaca_20k.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/20022 [00:00<?, ? examples/s]

In [12]:
dataset

Dataset({
    features: ['output', 'instruction', 'input'],
    num_rows: 20022
})

In [13]:
dataset[0]

{'output': 'arr = [2, 4, 6, 8, 10]',
 'instruction': 'Create an array of length 5 which contains all even numbers between 1 and 10.',
 'input': ''}

In [14]:
def format_dataset(example):
    # Combine instruction and input if input exists
    if example.get("input") and example["input"].strip():
        user_content = f"{example['instruction']}\n\nContext:\n{example['input']}"
    else:
        user_content = example["instruction"]

    return {
        "messages": [
            {"role": "user", "content": user_content},
            {"role": "assistant", "content": example["output"]}
        ]
    }

In [15]:
dataset = dataset.map(format_dataset)

Map:   0%|          | 0/20022 [00:00<?, ? examples/s]

In [16]:
messages = dataset[0]["messages"]
messages

[{'content': 'Create an array of length 5 which contains all even numbers between 1 and 10.',
  'role': 'user'},
 {'content': 'arr = [2, 4, 6, 8, 10]', 'role': 'assistant'}]

In [17]:
tokenizer = AutoTokenizer.from_pretrained(repo_id, trust_remote_code=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = "<|endoftext|>"
    tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids("<|endoftext|>")

tokenizer.padding_side = "right"

print(f"Pad Token: {tokenizer.pad_token}") # Should be <|endoftext|>
print(f"EOS Token: {tokenizer.eos_token}") # Should be <|im_end|>

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Pad Token: <|endoftext|>
EOS Token: <|im_end|>


In [18]:
tokenizer.chat_template

'{%- if tools %}\n    {{- \'<|im_start|>system\\n\' }}\n    {%- if messages[0][\'role\'] == \'system\' %}\n        {{- messages[0][\'content\'] }}\n    {%- else %}\n        {{- \'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.\' }}\n    {%- endif %}\n    {{- "\\n\\n# Tools\\n\\nYou may call one or more functions to assist with the user query.\\n\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>" }}\n    {%- for tool in tools %}\n        {{- "\\n" }}\n        {{- tool | tojson }}\n    {%- endfor %}\n    {{- "\\n</tools>\\n\\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\\n<tool_call>\\n{\\"name\\": <function-name>, \\"arguments\\": <args-json-object>}\\n</tool_call><|im_end|>\\n" }}\n{%- else %}\n    {%- if messages[0][\'role\'] == \'system\' %}\n        {{- \'<|im_start|>system\\n\' + messages[0][\'content\'] + \'<|im_end|>\\n\' }}\n    {%- else %}\n       

In [19]:
print(tokenizer.apply_chat_template(messages, tokenize=False))

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Create an array of length 5 which contains all even numbers between 1 and 10.<|im_end|>
<|im_start|>assistant
arr = [2, 4, 6, 8, 10]<|im_end|>



In [25]:
sft_config = SFTConfig(
    ## GROUP 1: Memory usage
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant': False},

    # BATCH SIZE ADJUSTMENT
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,  # 2 * 8 = 16 effective batch size
    auto_find_batch_size=True,      # Safety net: if 2 crashes, it tries 1

    ## GROUP 2: Dataset-related
    max_length=2048,
    packing=True,
    packing_strategy='wrapped',

    ## GROUP 3: Training Parameters
    num_train_epochs=1,
    learning_rate=5e-5,

    optim='paged_adamw_8bit',

    ## GROUP 4: Logging parameters
    logging_steps=5,
    logging_dir='./logs',
    output_dir='./qwen2.5-3b-coder-alpaca',
    report_to='tensorboard',

    # Precision logic
    bf16=torch.cuda.is_bf16_supported(including_emulation=False),
    fp16=not torch.cuda.is_bf16_supported(including_emulation=False)
)

In [26]:
trainer = SFTTrainer(
    model=model.base_model.model, # the underlying model
    peft_config=config,
    processing_class=tokenizer,
    args=sft_config,
    train_dataset=dataset,
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [27]:
dl = trainer.get_train_dataloader()
batch = next(iter(dl))

In [28]:
batch['input_ids'][0], batch['labels'][0]

(tensor([  803, 11050,    11,  ...,   366,   376,   397], device='cuda:0'),
 tensor([  803, 11050,    11,  ...,   366,   376,   397], device='cuda:0'))

In [29]:
trainer.train()

Step,Training Loss
5,1.364900
10,0.977600
15,0.758300


Step,Training Loss
5,1.364900
10,0.977600
15,0.758300
20,0.708700
25,0.695400
30,0.672500
35,0.688800
40,0.678500
45,0.650400
50,0.653100


TrainOutput(global_step=70, training_loss=0.7472982270377023, metrics={'train_runtime': 6426.4764, 'train_samples_per_second': 0.172, 'train_steps_per_second': 0.011, 'total_flos': 3.791404697596723e+16, 'train_loss': 0.7472982270377023})

In [30]:
def gen_prompt(tokenizer, instruction, input_context=""):
    """
    Generates a properly formatted chat prompt for Qwen.

    Args:
        tokenizer: The loaded Qwen tokenizer.
        instruction: The main user question (e.g., "Write a Python function...").
        input_context: (Optional) Code snippets or data to help answer the question.
    """

    if input_context:
        full_content = f"{instruction}\n\nContext:\n{input_context}"
    else:
        full_content = instruction

    messages = [{"role": "user", "content": full_content}]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    return prompt

In [41]:
sentence = "Write a python script to scrape a url."
prompt = gen_prompt(tokenizer, sentence)

In [42]:
print(prompt)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Write a python script to scrape a url.<|im_end|>
<|im_start|>assistant



In [43]:
def generate(model, tokenizer, prompt, max_new_tokens=2048, skip_special_tokens=False):
    tokenized_input = tokenizer(
        prompt,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(model.device)

    model.eval()

    ctx = torch.autocast(device_type=model.device.type, dtype=model.dtype) \
          if model.dtype in [torch.float16, torch.bfloat16] else torch.no_grad()
    with ctx:
        gen_output = model.generate(**tokenized_input,
                                    eos_token_id=tokenizer.eos_token_id,
                                    max_new_tokens=max_new_tokens,
                                    pad_token_id=tokenizer.pad_token_id,

                                    do_sample=True,
                                    temperature=0.2,
                                    top_p=0.95)

    input_length = tokenized_input.input_ids.shape[1]
    new_tokens = gen_output[0][input_length:]

    output = tokenizer.batch_decode(gen_output, skip_special_tokens=skip_special_tokens)
    return output[0]

In [44]:
print(generate(model, tokenizer, prompt))

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Write a python script to scrape a url.<|im_end|>
<|im_start|>assistant
Sure! Below is an example of a Python script that uses the `requests` library to scrape a URL and extract data from it:

```python
import requests

# Define the URL you want to scrape
url = "https://www.example.com"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    html_content = response.text
    
    # Use BeautifulSoup to parse the HTML content
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract data from the parsed HTML content
    # For example, extract all the links on the page
    links = soup.find_all('a')
    for link in links:
        print(link.get('href'))
else:
    print("Failed to retrieve the page")
```

This

In [45]:
trainer.save_model('local-qwen2.5-3b-coder-alpaca-adapter')

In [36]:
trainer.push_to_hub()

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...n2.5-3b-coder-alpaca/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            

  ...er-alpaca/adapter_model.safetensors:   0%|          | 60.2kB / 59.9MB            

  ...5-3b-coder-alpaca/training_args.bin:   1%|          |  58.0B / 6.10kB            

CommitInfo(commit_url='https://huggingface.co/bilalburney/qwen2.5-3b-coder-alpaca/commit/3f68e057f69c3e3d60dd9c4f59f19fe38ee2e1ef', commit_message='End of training', commit_description='', oid='3f68e057f69c3e3d60dd9c4f59f19fe38ee2e1ef', pr_url=None, repo_url=RepoUrl('https://huggingface.co/bilalburney/qwen2.5-3b-coder-alpaca', endpoint='https://huggingface.co', repo_type='model', repo_id='bilalburney/qwen2.5-3b-coder-alpaca'), pr_revision=None, pr_num=None)

In [38]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
drive_output_dir = "/content/drive/My Drive/qwen2.5-3b-coder-alpaca"
os.makedirs(drive_output_dir, exist_ok=True)

In [40]:
trainer.save_model(drive_output_dir)
tokenizer.save_pretrained(drive_output_dir)

print(f"Model and tokenizer successfully saved to: {drive_output_dir}")

Model and tokenizer successfully saved to: /content/drive/My Drive/qwen2.5-3b-coder-alpaca
